In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import json
from random import uniform
import time

In [2]:
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [3]:
urll = 'https://data.iros.go.kr/cr/rs/selectRgsCsTab.do'

In [4]:
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '203',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Cookie': 'WMONID=VdzGSVmMDjU; stoken=Vy3zFySFx5FAS1zTyGIDx5FDEMO1zCy1629212191zPy86400zAy33zEyXshPwJiufQjx78uqKWSB99x7A70EbrDx78ZeUo1cM63aDYQ3eOiKcPEtj2bMVudHF1O3Tx78x2BcKx2FBmfPx2FcpWJlx2B9DRPtFgx3Dx3DzKyUFSx796x79tjLx2FQNb9ht9FM1582XSHYx2FfvjNHuVShcgZfHhQWWnOnSIYgEH71x2F2jQ1x7ArzSSy00003097071zUURy480e8c733be2bbc22b3fba072a3941dbzMyXR0P1SScDQox3Dz; JSESSIONID=VMsOwjDn06iu1xwFH3Er4F66DfWn3u3n0z2aYyPgawy1t3TTpe59mlXlm24HbAaC.RF9vdXRfZG9tYWluMS9nNDQwTTE=',
    'Host': 'data.iros.go.kr',
    'Origin': 'https://data.iros.go.kr',
    'Referer': 'https://data.iros.go.kr/cr/rs/selectRgsCsDetl.do',
    'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
    'sec-ch-ua-mobile': '?0',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

In [5]:
date_list = []
for yy in range(2012,2022):
    for mm in range(1,13):
        if len(str(mm)) == 1:
            mm_str = '0' + str(mm)
            date_list.append(str(yy)+str(mm_str))
        else:
            date_list.append(str(yy)+str(mm))
        
        if yy==2021 and mm==7:
            break

In [6]:
print(date_list[0])
print(date_list[len(date_list)-1])

201201
202107


In [7]:
data = {
    'rdata_seq' : '0000000048',
    'search_real_cls': '13321',
    'search_sql': 'm01',
    'search_type': '02',
    'search_start_date': '202102',
    'search_end_date': '202102',
    'search_regn_cls': '02',
    'search_tab': 'grid',
    'search_regn_name': '',
    'search_bdata_age': '',
    'search_sex': ''
}

In [13]:
df_list = []
for rdata_seq in ['0000000053','0000000048']:
    for dd in date_list:
        time.sleep(uniform(1,3))
        for real_cls_code in ['','13311','13321','13323']: # 전체, 토지, 건물, 집합건물 순.
            data['rdata_seq'] = rdata_seq
            data['search_real_cls'] = real_cls_code
            data['search_start_date'] = dd
            data['search_end_date'] = dd

            time.sleep(uniform(1,2))

            reqq_text = requests.post(urll,headers=headers,data=data).text
            tmp_df    = pd.DataFrame(json.loads(reqq_text)['dataList'])
            
            tmp_df['rdata_seq'] = rdata_seq
            tmp_df['search_real_cls'] = real_cls_code
            tmp_df['search_start_date'] = dd
            tmp_df['search_end_date'] = dd

            time.sleep(uniform(1,3))

            df_list.append(tmp_df)

In [14]:
df_bind = pd.concat(df_list).reset_index(drop=True)
df_bind

,admin_regn1_name,bdata_age,sex,admin_regn2_name,tot,admin_regn1_cd,admin_regn2_cd,sex_cd,res_date,rdata_seq,search_real_cls,search_start_date,search_end_date
0,서울특별시,19세~29세,여자,강남구,1,11,강남구,F,2012-01,0000000053,,201201,201201
1,서울특별시,30세~39세,여자,강남구,4,11,강남구,F,2012-01,0000000053,,201201,201201
2,서울특별시,50세~59세,여자,강남구,4,11,강남구,F,2012-01,0000000053,,201201,201201
3,서울특별시,70세 이상,여자,강남구,1,11,강남구,F,2012-01,0000000053,,201201,201201
4,서울특별시,40세~49세,남자,강남구,2,11,강남구,M,2012-01,0000000053,,201201,201201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544212,경상남도,40세~49세,남자,창원시 의창구,1,48,창원시 의창구,M,2021-07,0000000048,13323,202107,202107
544213,경상남도,50세~59세,남자,창원시 진해구,1,48,창원시 진해구,M,2021-07,0000000048,13323,202107,202107
544214,제주특별자치도,50세~59세,여자,제주시,1,50,제주시,F,2021-07,0000000048,13323,202107,202107
544215,제주특별자치도,19세~29세,남자,제주시,1,50,제주시,M,2021-07,0000000048,13323,202107,202107


In [15]:
df_bind.to_csv('C:/Users/rsh15/Google Drive/crawler_data/estate_registration/auction_count_by_age.csv',index=False,encoding='UTF-8-sig')